## READ ME
### To run this notebook, you will need to specify the path to 1. A time series of ground-truth and predicted bounding box frames, concatenated as [first-5-ground, 2nd-5-ground, predicted-5], 2. The path to the binary lidar data files for this set of frames

In [1]:
import numpy as np
import open3d as o3d
import time
import os

In [ ]:
BOX_FRAMES_PATH = None #TODO
LIDAR_FRAMES_PATH = None #TODO

## Center the predicted and true bounding boxes in the lidar space

In [2]:
def box_center_to_corner(box, pred=False):
    """
    Centers a bounding box in lidar data; pass pred=True if it is a predicted bounding box
    """
    h,w,l,x,y,z,rotation = box
    h,w,l,x,y,z,rotation = box
    bounding_box = np.array([
        [l/2, l/2, -l/2, -l/2, l/2, l/2, -l/2, -l/2],
        [0,0,0,0,-h,-h,-h,-h],
        [w/2, -w/2, -w/2, w/2, w/2, -w/2, -w/2, w/2]])

    # Standard 3x3 rotation matrix around the Z axis
    rotation_matrix = np.array([
        [np.sin(rotation), np.cos(rotation), 0.0],
        [np.cos(rotation), -np.sin(rotation), 0.0],
        [0.0, 0.0, 1.0]])
    #corner_box = np.dot(rotation_matrix, bounding_box).T
    corner_box = bounding_box.T
    if not pred:
        corner_box[:,0] -= (x - 15)
        corner_box[:,1] -= (y - 4.5)
        corner_box[:,2] -= 0.5
    else:
        corner_box[:,0] -= (x - 10)
        corner_box[:,1] -= (y - 4.5)
        corner_box[:,2] -= 0.5
    return corner_box

In [3]:

def print_camera_pose(vis):
    """Writes the camera orientation of the Open3D visualization window
    Use shift + A to record position in open3D render window
    """
    #ctr = vis.get_view_control()
    cam_params = ctr.convert_to_pinhole_camera_parameters()
    with open('cam.txt', 'w') as f:
        f.write(f"Look: {cam_params.extrinsic[:3,2]}, Up: {cam_params.extrinsic[:3,3]}, Front: {cam_params.extrinsic[:3,1]},something: {cam_params.extrinsic[:3,0]}")
        #f.write(f"lookat={ctr.get_lookat_point()} front={ctr.get_front_vector()} up={ctr.get_up_vector()} zoom={ctr.get_zoom()}")
        f.write('\n')
        #print(f"Look: {cam_params.extrinsic[:3,2]}, Up: {cam_params.extrinsic[:3,3]}, Front: {cam_params.extrinsic[:3,1]}")

In [4]:
def vis_box_cloud(box, bin_file, true_box=None):
    """
    Visualizes the bounding box(es) in the lidar data. 
    Takes a binary lidar data file and a 1 or 2 bounding boxes
    """
    #box = box_center_to_corner(box)
    bin_pcd = np.fromfile(bin_file, dtype=np.float32)
    points = bin_pcd.reshape((-1, 4))[:, 0:3]
    # Convert to Open3D point cloud
    o3d_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))
    # Save to whatever format you like
    pcd_file = f"pcd/{bin_file[5:-4]}.pcd"
    o3d.io.write_point_cloud(pcd_file, o3d_pcd)
    pcd = o3d.io.read_point_cloud(pcd_file)
    lines = [[0, 1], [1, 2], [2, 3], [0, 3],
          [4, 5], [5, 6], [6, 7], [4, 7],
          [0, 4], [1, 5], [2, 6], [3, 7]]

    # Use the same color for all lines
    if type(true_box) != type(None):
        colors = [[1, 0, 0] for _ in range(len(lines))]
        true_colors = [[0, 0, 0] for _ in range(len(lines))]
        box = box_center_to_corner(box, pred=True)
    else:
        colors = [[0, 0, 0] for _ in range(len(lines))]
        box = box_center_to_corner(box)

    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(box)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)

    if type(true_box) != type(None):
        true_box = box_center_to_corner(true_box)
        true_lines = o3d.geometry.LineSet()
        true_lines.points = o3d.utility.Vector3dVector(true_box)
        true_lines.lines = o3d.utility.Vector2iVector(lines)
        true_lines.colors = o3d.utility.Vector3dVector(true_colors)
    #vis = o3d.visualization.Visualizer()
    #vis.add_geometries([line_set,pcd])
    if type(true_box) != type(None):
        o3d.visualization.draw_geometries([line_set,pcd,true_lines],
                                        zoom=0.1,
                                        lookat=[-0.04318952, -0.89840303, -0.43704309],
                                        up=[-0.59718197, -0.67550043, 27.44612798], 
                                        front=[-0.98925943, -0.02268793,  0.14439891]
                                        )
    else:
        o3d.visualization.draw_geometries([line_set,pcd],
                                        zoom=0.1,
                                        lookat=[ 0.03398664,-0.85294982,-0.52088531],
                                        up=[-0.53002478,-0.72589837,31.23243209], 
                                        front=[ 0.19140022, -0.43721607,  0.8787537 ]
                                        )
        # Comment the above block and uncomment below to sue shit+A to record position of render 
        # o3d.visualization.draw_geometries_with_key_callbacks([line_set,pcd],{ord('A'):print_camera_pose})




    

In [5]:
first_5_data, ground_data, pred_data = np.load(BOX_FRAMES_PATH)

## Visualize an example

In [6]:
i = 0
files = sorted(os.listdir(LIDAR_FRAMES_PATH))
binfile = LIDAR_FRAMES_PATH+files[0]
print(binfile)
true_box = None
if i < 5:
    box = first_5_data[i]
else:
    box = pred_data[i-5]
    true_box = ground_data[i-5]
vis_box_cloud(box,binfile,true_box)
